# 인터페이스
자바는 정적 타입 언어다.
그래서 타입을 드러나 보이게 작성하지 않고 `var`로 변수를 선언하는 경우에도 컴파일러는 각각의 변수가 무슨 타입인지 추론한다.

레코드와 같은 여러 사용자 정의 타입을 함께 섞어 일관된 방식으로 사용하려면
그런 여러 타입을 아우르는 공통된 타입을 선언해야 하는데, 이럴 때 활용할 수 있는 것이 바로 **인터페이스**(interface)다.

## 문제 상황
아래와 같이 두 레코드 `Circle`와 `Rectangle`에
동일한 접근 이름, 제어 권한, 리턴 타입, 파라메터를 지닌 메소드 `area()`가 있다고 하자.

In [1]:
record Circle(int radius) {
  public double area() {
    return Math.PI * radius * radius;
  }
}

record Rectangle(int width, int height) {
  public double area() {
    return width * height;
  }
}

그리고 각각의 인스턴스를 원소로 포함하는 리스트를 이렇게 만들어서

In [2]:
var figures = List.of(new Circle(2), new Rectangle(3, 4));

아래와 같이 반복문을 통해 리스트의 각 원소의 넓이를 `area()` 메소드를 호출하여 알아보려 하면 컴파일되지 않는다.

In [3]:
for(var figure: figures) {
  System.out.println(figure.area());
}

CompilerException: 

왜냐하면 `Circle`와 `Rectangle`를 공통으로 아우르는 타입은 `java.lang.Object` 뿐인데,
이 `Object` 타입에는 `area()`라는 메소드가 정의되어 있지 않기 때문이다.

### 인터페이스(interface)와 추상 메소드(abstract method)

문제를 해결하는 방법은 `Circle`와 `Rectangle`의 공통된 타입이면서 `area()` 메소드도 호출할 수 있도록 하는 것이다.
자바에서는 이러한 공통된 타입을 `interface` 키워드를 활용해 선언할 수 있다.

아래 `Figure`의 `area()` 메소드는 지금까지 보아 왔던 `Circle`나 `Rectangle`의 메소드처럼
함수 몸체의 코드가 포함된 메소드와는 달리 함수 몸체에 해당하는 내용이 없다.

In [4]:
interface Figure {
    public abstract double area();
    // double area(); // 이렇게 public과 abstract를 생략하더라도 기본적으로 public abstract
}

위와 같은 메소드를 추상(`abstract`) 메소드라고 하며,
메소드의 형식(이름, 접근 권한, 리턴 타입, 파라메터)만 선언되어 있다.

구체적인 실행 내용에 해당하는 부분(함수 몸체)은 인터페이스를 구현하는 각각의 레코드에서 구현되어야 한다.
아래와 같이 `implements` 키워드를 활용하여 `Circle`와 `Rectangle` 레코드를 `Figure` 인터페이스를 구현하는 레코드라고 선언할 수 있다.

In [5]:
record Circle(int radius) implements Figure {
    public double area() {
        return Math.PI * radius * radius;
    }
}

record Rectangle(int width, int height) implements Figure {
    public double area() {
        return width * height;
    }
}

이제는 아무런 다른 정보 없이 그냥 객체(Object)라는 공통점만이 아닌, `Figure`라는 공통된 타입의 리스트(`List<Figure>`)로 처리하면 반복문을 순회하며 `Circle`나 `Rectangle`의 인스턴스인 리스트의 원소에 대해 `area()`를 호출하는 코드가 타입 오류 없이 컴파일되어 실행된다.

In [20]:
List<Figure> figures = List.of(new Circle(2), new Rectangle(3, 4));

for(var figure: figures) {
  System.out.println(figure.area());
}

4.0
12.0


인터페이스란 서로 다른 레코드에서 동일한 메소드를 호출하려는 경우 선언하여 활용하는 공통의 타입이다.
실행 시간(runtime)에 인터페이스의 메서드를 호출하면 자바 가상 머신이 올바른 구현을 동적으로 연결(dynamic binding)하여 인스턴스의 타입에 맞는 적절한 내용의 메소드를 실행한다. 이를 하위타입 다형성(subtype polymorphism)이라고 한다.

## 클래스 메소드
Like a record, an interface can have `static` methods


In [7]:
interface Figure {
  public abstract double area();
  public static Figure createACircle(int radius) {
    return new Circle(radius);
  }
}

In [8]:
var aCircle = Figure.createACircle(3);

System.out.println(aCircle);

Circle[radius=3]


## Default method
Inradius an interface, the instance methods are implicitly abstract,
if we want to declare a method with some code in it, we have to use
the keyword `default`.
By example, we can write a method `isBig` that is true if the area is big enough.


In [9]:
interface Figure {
  public abstract double area();
  public default boolean isBig() {
    return area() >= 10;
  }
}

In [10]:
record Circle(int radius) implements Figure {
  public double area() {
    return radius * radius;
  }
}

record Rectangle(int width, int height) implements Figure {
  public double area() {
    return width * height;
  }
}

In [11]:
System.out.println(new Circle(2).isBig());
System.out.println(new Rectangle(3, 4).isBig());

false
true


Because a default method is declared on the interface, all records that
implement that interface will have that method. Default methods are named like this
because if a record that implements the interface doesn't define the method,
the method will be provided by default.


## 함수형 인터페이스
추상 함수가 딱 하나만으로 이루어진 인터페이스는 의미상 함수의 타입과도 같다.
이런 경우를 **함수형 인터페이스**(functional_ interface)라고 부르며,
앞서 살펴본 `implements` 키워드를 활용하는 일반적인 구현 방식 외에
또 다른 두 가지 특별한 방식으로 구현할 수 있다.

### Lambda
자바의 람다식은 괄호 안에 파라메터를 작성하고 `->`를 작성한 다음 리턴값 식 또는 중괄호로 작성하는 함수 몸체를 작성한다.

In [12]:
interface Figure {
  public abstract double area();
}

Figure anotherFigure = () -> 4;

System.out.println(anotherFigure.area());

4.0


and rewrite the method rectangularTriangle()
You can notice that a lambda can access to the parameter `width` and `height`


In [13]:
Figure rectangularTriangle(int width, int height) {
  return () -> width * height / 2.0;
}
var triangle = rectangularTriangle(3, 4);
System.out.println(triangle.area());


6.0


### 메소드 참조 method reference
In case of the method already exists instead of 
calling it inradius a lambda, we can make a reference on it using the operator `::`
(notice that EquilaterlaTriangle doesn't implement Figure)


In [14]:
record EquilateralTriangle(int radius) {
  double area() {
    return Math.sqrt(3) * radius * radius / 4.0;
  }
}

var equilateral = new EquilateralTriangle(2);

so instead of


In [15]:
var figures = List.<Figure>of(new Circle(2), () -> equilateral.area());

for(var figure: figures) {
  System.out.println(figure.area());
}

4.0
1.7320508075688772


you can use a method reference


In [25]:
var figures = List.<Figure>of(new Circle(2), equilateral::area);

for(var figure: figures) {
  System.out.println(figure.area());
}

4.0
1.7320508075688772


More about lambdas and method references in the following chapter.
